# Install packages

In [ ]:
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install osmnet
!pip install osmnet
!pip install pandana
!pip install folium

In [ ]:
!git clone https://github.com/CityScope/CSL_HCMC

fatal: destination path 'CSL_HCMC' already exists and is not an empty directory.


# Import lib

In [ ]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
from collections import Counter

# Get dataset

In [ ]:
data = gpd.read_file("/content/CSL_HCMC/Data/GIS/Population/population_HCMC/population_shapefile/Population_District_Level.shp")

In [ ]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Dist_Name   24 non-null     object  
 1   Dist_ID     24 non-null     object  
 2   Level       24 non-null     object  
 3   Pop_2009    24 non-null     float64 
 4   Pop_2010    24 non-null     float64 
 5   Pop_2011    24 non-null     float64 
 6   Pop_2012    24 non-null     float64 
 7   Pop_2013    24 non-null     float64 
 8   Pop_2014    24 non-null     float64 
 9   Pop_2015    24 non-null     float64 
 10  Pop_2016    24 non-null     float64 
 11  Pop_2017    24 non-null     float64 
 12  Pop_2019    24 non-null     float64 
 13  Den_2009    24 non-null     float64 
 14  Den_2010    24 non-null     float64 
 15  Den_2011    24 non-null     float64 
 16  Den_2012    24 non-null     float64 
 17  Den_2013    24 non-null     float64 
 18  Den_2014    24 non-null     float64 
 19  De

In [ ]:
data = data[['Dist_Name', 'Dist_ID', 'Pop_2017', 'Pop_2019', 'geometry']]
data['Pop_ratio'] = data['Pop_2019'] / data['Pop_2017']
sorted_data = data.sort_values(by='Pop_ratio', axis=0, ascending=False)
sorted_data = sorted_data.reset_index()
sorted_data.head()

data1 = sorted_data.iloc[:10, 1:]
data1

,Dist_Name,Dist_ID,Pop_2017,Pop_2019,Shape_Leng,Shape_Area,geometry,Pop_ratio
0,District 9,763,310739.0,397006.0,53139.220230,1.142649e+08,"POLYGON ((701125.957 1205414.891, 701162.391 1...",1.277619
1,Hoc Mon,784,446056.0,542243.0,65955.793966,1.091744e+08,"POLYGON ((673061.206 1208548.077, 673094.550 1...",1.215639
2,Nha Be,786,177990.0,206837.0,67540.905029,1.004901e+08,"POLYGON ((688444.826 1185726.125, 688568.759 1...",1.162071
3,District 12,761,549064.0,620146.0,47703.394094,5.277342e+07,"POLYGON ((685183.891 1205265.848, 685198.395 1...",1.129460
4,District 2,769,161957.0,180275.0,39592.970451,4.993869e+07,"POLYGON ((690496.729 1195354.084, 690716.888 1...",1.113104
5,Cu Chi,783,418655.0,462047.0,119954.769999,4.346845e+08,"POLYGON ((656983.014 1228510.664, 656969.299 1...",1.103646
6,District 7,778,327223.0,360155.0,33260.915360,3.525220e+07,"POLYGON ((691506.715 1190058.151, 691359.893 1...",1.100641
7,Thu Duc,762,542612.0,592686.0,52828.501430,4.775581e+07,"POLYGON ((688796.063 1204383.598, 688818.721 1...",1.092283
8,Binh Chanh,785,652900.0,705508.0,129933.645113,2.529152e+08,"POLYGON ((679646.388 1186348.769, 679698.864 1...",1.080576
9,Binh Tan,777,729366.0,784173.0,40057.584063,5.191097e+07,"POLYGON ((674953.073 1197210.125, 674961.146 1...",1.075143


In [ ]:
data1 = data1.to_crs(32648)

In [ ]:
data2 = gpd.read_file('/content/response3.json')
data2 = data2.to_crs(32648)

In [ ]:
data2.crs

<Projected CRS: EPSG:32648>
Name: WGS 84 / UTM zone 48N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 102°E and 108°E, northern hemisphere between equator and 84°N, onshore and offshore. Cambodia. China. Indonesia. Laos. Malaysia - West Malaysia. Mongolia. Russian Federation. Singapore. Thailand. Vietnam.
- bounds: (102.0, 0.0, 108.0, 84.0)
Coordinate Operation:
- name: UTM zone 48N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
data2

,dbh,geometry
0,1,POINT (106.71136 10.85113)
1,1,POINT (106.71695 10.74449)
2,1,POINT (106.66315 10.83289)
3,1,POINT (106.66267 10.83339)
4,1,POINT (106.65977 10.83729)
...,...,...
83670,1,POINT (106.65664 10.84339)
83671,1,POINT (106.75180 10.85276)
83672,1,POINT (106.73189 10.73154)
83673,1,POINT (106.67640 10.82388)


In [ ]:
merged = gpd.sjoin(data2, data1,how='left', op='within')
merged.dropna(axis=0, how='any', subset=['Dist_ID'], inplace=True)

In [ ]:
merged = merged.sort_values(by='Pop_ratio', axis=0, ascending=False)
merged = merged.reset_index()
merged = merged.iloc[:, 2:]
merged

,geometry,index_right,Dist_Name,Dist_ID,Pop_2017,Pop_2019,Shape_Leng,Shape_Area,Pop_ratio
0,POINT (700374.950 1198770.525),0.0,District 9,763,310739.0,397006.0,53139.220230,1.142649e+08,1.277619
1,POINT (699456.443 1204720.624),0.0,District 9,763,310739.0,397006.0,53139.220230,1.142649e+08,1.277619
2,POINT (698836.877 1192704.024),0.0,District 9,763,310739.0,397006.0,53139.220230,1.142649e+08,1.277619
3,POINT (700309.979 1199276.144),0.0,District 9,763,310739.0,397006.0,53139.220230,1.142649e+08,1.277619
4,POINT (694671.602 1197600.447),0.0,District 9,763,310739.0,397006.0,53139.220230,1.142649e+08,1.277619
...,...,...,...,...,...,...,...,...,...
45193,POINT (675285.552 1192572.669),9.0,Binh Tan,777,729366.0,784173.0,40057.584063,5.191097e+07,1.075143
45194,POINT (675023.642 1195415.642),9.0,Binh Tan,777,729366.0,784173.0,40057.584063,5.191097e+07,1.075143
45195,POINT (676026.205 1194298.105),9.0,Binh Tan,777,729366.0,784173.0,40057.584063,5.191097e+07,1.075143
45196,POINT (676240.723 1194210.240),9.0,Binh Tan,777,729366.0,784173.0,40057.584063,5.191097e+07,1.075143


In [ ]:
dists = merged['Dist_ID'].unique()
dists

array(['763', '784', '786', '761', '769', '783', '778', '762', '785',
       '777'], dtype=object)

# Create HeatMap

In [ ]:
from folium import plugins
from folium.plugins import HeatMap
m = folium.Map(location=[10.769444, 106.681944], zoom_start=12, tiles='CartoDB positron')

In [45]:
def get_coor(p):
  return np.array([float(p.x), float(p.y)], dtype=np.float32)
  
for dist in dists:
  temp = merged[merged['Dist_ID'] == dist]
  X = np.array(list(temp['geometry'].map(get_coor)))
  model = KMeans(n_clusters=20, random_state=0)
  model.fit(X)

  counter = Counter(model.labels_)
  largest_cluster = np.argmax(counter.values())
  largest_cluster_label = model.labels_[largest_cluster]
  pos_largest_cluster = X[model.labels_ == largest_cluster]
  heatm = pd.DataFrame(points_in_largest_cluster, columns=['Latitude', 'Longtitude'], dtype=np.float32)
  heat_data = [[row['Longtitude'],row['Latitude']] for index, row in heatm.iterrows()]
  HeatMap(heat_data).add_to(m)

m